In [1]:
import os
import pandas as pd
import numpy as np
import pickle
from pandas import HDFStore

import yaml

cfg = None
    #' load config.yaml file in the root dir 
with open("../config.yaml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)
    
# Use 3 decimal places in output display
pd.set_option("display.precision", 3)

# Don't wrap repr(DataFrame) across additional lines
pd.set_option("display.expand_frame_repr", False)

# Set max rows displayed in output to 25
pd.set_option("display.max_rows", 25)

pd.set_option('display.float_format', lambda x: '%.2f' % x)


#Function to get data frame summary
def get_data_frame_summmary(data_frame, sort_by='percent_missing', ascending=False):
    unique_values = data_frame.apply(lambda x: [x.unique()])
    unique_counts = data_frame.apply(lambda x: len(x.unique()))
    percent_missing = data_frame.apply(lambda x: sum(pd.isnull(x))/len(x)*100)
    data_type = data_frame.dtypes

    return pd.DataFrame(dict(unique_values = unique_values, 
                                unique_counts = unique_counts,
                                data_type = data_type,
                                percent_missing = percent_missing,
                                )).reset_index().sort_values(by=sort_by, ascending=ascending)

# Function to drop outliers of numeric columns
def drop_outliers(data_frame, exclude=[]):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    num_cols = data_frame.select_dtypes(include=numerics).columns

    num_cols = num_cols.difference(exclude)
    
    for colname in num_cols:
        upper_lim = data_frame[colname].quantile(.95)
        lower_lim = data_frame[colname].quantile(.05)
        
        print(f'Dropping outliers for {colname} upper limit = {upper_lim} and lower limit = {lower_lim}')
        data_frame = data_frame[(data_frame[colname] < upper_lim) & (data_frame[colname] > lower_lim)]
      
    return data_frame


# Function to drop outliers of numeric columns
def scale_numeric_features(data_frame, exclude=[], 
                           method='standardize',
                           inplace=False):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    num_cols = data_frame.select_dtypes(include=numerics).columns

    num_cols = num_cols.difference(exclude)
    print(f'********************* - Scaling following {len(num_cols)} features - **********************')
    for colname in num_cols:
        new_colname = colname if inplace else colname+'_'+method+'d'
        print(f' {colname} {method}d to {new_colname}')
        
        if method == 'standardize':              
              data_frame[new_colname] = (data_frame[colname] - data_frame[colname].mean()) / data_frame[colname].std()
        elif method == 'normalize':
              data_frame[new_colname] = (data_frame[colname] - data_frame[colname].min()) / (data_frame[colname].max() - data_frame[colname].min())
        else:
              print(f'Unknown method {method} specified, please select one of "standardize" or "normalize"')

      
    return data_frame


def remove_items(iteamlist, removelist):
    return [ele for ele in iteamlist if ele not in removelist]

def diff(first, second):
    second = set(second)
    return [item for item in first if item not in second]
    
def months_between_columns(date_series1, date_series2):
    return round((date_series1-date_series2) / np.timedelta64(1, 'M'))
    
path_to_hdf_datastore = cfg['path_to_hdf_datastore']
path_to_clean_hdf_datastore = cfg['path_to_clean_hdf_datastore']

In [2]:
# Read Data
cancellations = pd.read_hdf(path_to_hdf_datastore, cfg['cancellations_file'])

df = cancellations
del cancellations

df.head()

,Source System,SIS Id (Agreement SIS),Source System.1,HQ SIS Id (Agreement SIS),Name (Agreement SIS),Business Division (Agreement SIS),Sales Division (Agreement SIS),Division,RSO,Subregion Grouping,...,Subscription Start Date,Subscription End Date,Parent Agreement Number,Currency(Entered),Revenue Final Net Price - Agent Discount Amount(Rep),Revenue Committed Print(Rep),Bookings - Final Net Price - Agent Discount Amount(Rep),Bookigns - Committed Print(Rep),Cancellation Month,Cancellation Reason
0,CRM,1060896,CRM,1060896,Hospital Infantil de las Californias,HS EMEA/LA,HS-EMEALA-RLAM0000000-01D,HS,EMEALA,HS RLAM,...,2015-05-15,2016-05-14,nan,USD,nan,nan,0.00,0.00,201702,Unknown
1,CRM,1060896,CRM,1060896,Hospital Infantil de las Californias,HS EMEA/LA,HS-EMEALA-RLAM0000000-01D,HS,EMEALA,HS RLAM,...,2015-05-15,2016-05-14,nan,USD,nan,nan,838.00,0.00,201702,Unknown
2,CRM,1060896,CRM,1060896,Hospital Infantil de las Californias,HS EMEA/LA,HS-EMEALA-RLAM0000000-01D,HS,EMEALA,HS RLAM,...,2015-05-15,2016-05-14,nan,USD,nan,nan,0.00,0.00,201702,Unknown
3,CRM,1060896,CRM,1060896,Hospital Infantil de las Californias,HS EMEA/LA,HS-EMEALA-RLAM0000000-01D,HS,EMEALA,HS RLAM,...,2015-05-15,2016-05-14,nan,USD,nan,nan,0.00,0.00,201702,Unknown
4,CRM,1060896,CRM,1060896,Hospital Infantil de las Californias,HS EMEA/LA,HS-EMEALA-RLAM0000000-01D,HS,EMEALA,HS RLAM,...,2015-05-15,2016-05-14,nan,USD,nan,nan,1.00,0.00,201702,Unknown


# Replace strings indicating missing data with null

In [3]:
#replace strings indicating missing data with null
df.replace(['nan', 'N.A', 'N.A.', 'NaN', 'Nan', '00-00-00', '0-00-00', 'Unknown'], np.nan, inplace=True)
df

,Source System,SIS Id (Agreement SIS),Source System.1,HQ SIS Id (Agreement SIS),Name (Agreement SIS),Business Division (Agreement SIS),Sales Division (Agreement SIS),Division,RSO,Subregion Grouping,...,Subscription Start Date,Subscription End Date,Parent Agreement Number,Currency(Entered),Revenue Final Net Price - Agent Discount Amount(Rep),Revenue Committed Print(Rep),Bookings - Final Net Price - Agent Discount Amount(Rep),Bookigns - Committed Print(Rep),Cancellation Month,Cancellation Reason
0,CRM,1060896,CRM,1060896,Hospital Infantil de las Californias,HS EMEA/LA,HS-EMEALA-RLAM0000000-01D,HS,EMEALA,HS RLAM,...,2015-05-15,2016-05-14,NaN,USD,nan,nan,0.00,0.00,201702,NaN
1,CRM,1060896,CRM,1060896,Hospital Infantil de las Californias,HS EMEA/LA,HS-EMEALA-RLAM0000000-01D,HS,EMEALA,HS RLAM,...,2015-05-15,2016-05-14,NaN,USD,nan,nan,838.00,0.00,201702,NaN
2,CRM,1060896,CRM,1060896,Hospital Infantil de las Californias,HS EMEA/LA,HS-EMEALA-RLAM0000000-01D,HS,EMEALA,HS RLAM,...,2015-05-15,2016-05-14,NaN,USD,nan,nan,0.00,0.00,201702,NaN
3,CRM,1060896,CRM,1060896,Hospital Infantil de las Californias,HS EMEA/LA,HS-EMEALA-RLAM0000000-01D,HS,EMEALA,HS RLAM,...,2015-05-15,2016-05-14,NaN,USD,nan,nan,0.00,0.00,201702,NaN
4,CRM,1060896,CRM,1060896,Hospital Infantil de las Californias,HS EMEA/LA,HS-EMEALA-RLAM0000000-01D,HS,EMEALA,HS RLAM,...,2015-05-15,2016-05-14,NaN,USD,nan,nan,1.00,0.00,201702,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353243,UPL,ECR-918266,UPL,ECR-918266,JSC National Center of Science and Technology ...,A&G_EMLA,AG-EMLA-Turmeca000000-04D,A&G,EMLA,Region Turmeca,...,2015-02-01,2016-01-31,NaN,USD,nan,nan,951134.48,0.00,201606,NaN
353244,UPL,ECR-928151,UPL,ECR-346544,National Institute of Food and Drug Safety Eva...,A&G_APAC,AG-APAC-Korea00000000-01D,A&G,A&G Vendor,A&G Vendor Sales,...,2016-12-31,2017-12-30,NaN,USD,nan,nan,11294.00,0.00,201703,NaN
353245,UPL,ECR-931550,UPL,ECR-931550,Republic of the Philippines Department of Health,A&G_APAC,AG-APAC-SEA0000000000-04D,A&G,A&G Vendor,A&G Vendor Sales,...,2017-01-01,2017-12-31,NaN,USD,nan,nan,11665.00,0.00,201705,NaN
353246,UPL,ECR-931550,UPL,ECR-931550,Republic of the Philippines Department of Health,A&G_APAC,AG-APAC-SEA0000000000-04D,A&G,A&G Vendor,A&G Vendor Sales,...,2019-01-02,2020-01-01,NaN,USD,12263.93,0.00,12297.62,0.00,201904,NaN


In [4]:
for i in range(0,df.shape[1]):
    print(df.iloc[:,i].value_counts())

CRM    353112
UPL       136
Name: Source System, dtype: int64
ECR-40004     2553
ECR-40026     2014
ECR-39962     1971
ECR-358312    1754
ECR-933990    1688
              ... 
ECR-890306       1
ECR-526567       1
ECR-635686       1
ECR-369517       1
ECR-502097       1
Name: SIS Id  (Agreement SIS), Length: 8586, dtype: int64
CRM    353112
UPL       136
Name: Source System.1, dtype: int64
ECR-876907    3658
ECR-40004     2553
ECR-40026     2014
ECR-39962     2004
ECR-358312    1754
              ... 
ECR-549110       1
ECR-39196        1
ECR-246541       1
ECR-960769       1
ECR-3000         1
Name: HQ SIS Id (Agreement SIS), Length: 7752, dtype: int64
Banaras Hindu University                          2553
Jawaharlal Nehru University                       2014
University of Delhi                               1971
Motilal Nehru National Institute of Technology    1754
National Health Insurance Administration          1688
                                                  ... 
Memorial

Nanomedicine: Nanotechnology, Biology and Medicine                                          684
World Patent Information                                                                    677
Computational Biology and Chemistry                                                         646
Journal of Molecular Graphics and Modelling                                                 643
Safety Science                                                                              637
                                                                                           ... 
UTL ORBIS J                                                                                   1
ClinicalKey SPA Medicina de Familia                                                           1
Insights Library 2017: Food Science â Access Plus                                           1
Insights Library 2015: Life and Biomedical Sciences â Access Plus                           1
MRW subscription package: International 

### Summary of the dataframe

In [5]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(get_data_frame_summmary(df))

,index,unique_values,unique_counts,data_type,percent_missing
36,Revenue Final Net Price - Agent Discount Amoun...,"[[nan, 813.84, 933.8622951, 373.62622949999997...",11288,float64,96.37
37,Revenue Committed Print(Rep),"[[nan, 0.0, 558.5, 657.75, 1382.64, 947.71, 65...",323,float64,89.26
38,Bookings - Final Net Price - Agent Discount Am...,"[[0.0, 838.0, 1.0, 915.8, 847.4, 1419.31, 1255...",47679,float64,82.14
22,Renewal Exp Complete Date,"[[nan, 2014-10-31, 2015-11-18, 2015-11-17, 201...",1143,object,48.71
41,Cancellation Reason,"[[nan, Account Management, Budget, Competitive...",10,object,31.05
19,Payment Term Type,"[[NET, AS_IS_DATE, nan, SPL/MTH]]",4,object,8.32
18,Payment Term Description,"[[Net Due in 30 days After Invoice Date, Net D...",86,object,8.32
17,Payment Term,"[[30 NET, 90 NET, 28FEB15, 60 NET, 15DEC18, na...",86,object,8.32
34,Parent Agreement Number,"[[nan, 1-4772884818, 1-5302903067, 1-103094846...",10597,object,0.32
15,Sales Type,"[[Cancelled, Active Renewal, nan]]",3,object,0.04


# Identify Missing Data

Missing values affect the performance of the machine learning models and its useful to identify and drop or impute missing values before modelling.

We will quantify the missing data and drop any columns lower than the given threshold. we have set threshold of 70% so any columns and rows with missing data over 70% are dropped.

In this case, manually ignore Renewal Exp Complete Date. We can also manually drop 'Payment Term Description' since it is highly correlated to 'Payment Term'.

In [6]:
#set threshold
threshold = 0.7


#Dropping columns with missing value rate higher than threshold
temp = df[df.columns[df.isnull().mean() < threshold]]

print('-------------------- Dropping Columns with missing data --------------------')
print(f'Following {len(df.columns.difference(temp.columns))} columns have missing data over the threshold and will be removed')
print(df.columns.difference(temp.columns))

df = temp

print('-------------------- Dropping rows with missing data --------------------')
print(f' There are {sum(df.isnull().mean(axis=1) > threshold)} rows with missing data over the threshold')

#Dropping rows with missing value rate higher than threshold
df = df.loc[df.isnull().mean(axis=1) < threshold]

# remove temp object from memory
del temp

#manually remove borderline case
df = df.drop(columns=['Renewal Exp Complete Date', 'Payment Term Description'])

-------------------- Dropping Columns with missing data --------------------
Following 3 columns have missing data over the threshold and will be removed
Index(['Bookings - Final Net Price - Agent Discount Amount(Rep)',
       'Revenue Committed Print(Rep)',
       'Revenue Final Net Price - Agent Discount Amount(Rep)'],
      dtype='object')
-------------------- Dropping rows with missing data --------------------
 There are 0 rows with missing data over the threshold


In [7]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(get_data_frame_summmary(df))

,index,unique_values,unique_counts,data_type,percent_missing
36,Cancellation Reason,"[[nan, Account Management, Budget, Competitive...",10,object,31.05
17,Payment Term,"[[30 NET, 90 NET, 28FEB15, 60 NET, 15DEC18, na...",86,object,8.32
18,Payment Term Type,"[[NET, AS_IS_DATE, nan, SPL/MTH]]",4,object,8.32
32,Parent Agreement Number,"[[nan, 1-4772884818, 1-5302903067, 1-103094846...",10597,object,0.32
15,Sales Type,"[[Cancelled, Active Renewal, nan]]",3,object,0.04
14,Business Indicator,"[[Cancelled, New Sale, Renewal, Credit, nan]]",5,object,0.00
3,HQ SIS Id (Agreement SIS),"[[1060896, 1072415, 2064, 304, 834326, 906206,...",7752,object,0.00
27,Agreement Number,"[[1-8619116948, 1-8204696302, 1-8810268492, 1-...",11339,object,0.00
23,Product Line Level 2,"[[ClinicalPharmacology, clinics, Journals Elec...",34,object,0.00
24,Product Line Level 3,"[[Clinical Pharmacology Access Codes, Clinical...",159,object,0.00


### Data Imputation

We can impute missing data with meaningful data so that model development has good quality of data

We can impute categorical variables with the most frequestly occuring value and impute numerical variables with 0 or mean or median depending on the variable context.

Cancellation Reason - Impute with most frequent

Payment Term - Impute with most frequent

Payment Term Type - Impute with most frequent


In [8]:
#Filling all missing values with 0
# data = data.fillna(0)

#Filling missing values with the most frequest values
df['Cancellation Reason'].fillna(
    df['Cancellation Reason'].value_counts().idxmax(), inplace=True)

df['Payment Term'].fillna(
    df['Payment Term'].value_counts().idxmax(), inplace=True)

df['Payment Term Type'].fillna(
    df['Payment Term Type'].value_counts().idxmax(), inplace=True)

# Handling Outliers

Extreme value can skew the data distribution and thus affect the model development we identify outliers in numeric variables and handle them by removing or capping.

### Outlier Detection with Percentiles

In [9]:
#Dropping the outlier rows with Percentiles

df = drop_outliers(df, exclude = ['Bookigns - Committed Print(Rep)'])

Dropping outliers for Cancellation Month upper limit = 201912.0 and lower limit = 201503.0
Dropping outliers for Reporting Year (2015) upper limit = 2015.0 and lower limit = 2015.0


# Binning

The main motivation of binning is to make the model more robust and prevent overfitting, however, it has a cost to the performance. Every time you bin something, you sacrifice information and make your data more regularized

The trade-off between performance and overfitting is the key point of the binning process


For Categorical variables the labels with low frequencies probably affect the robustness of statistical models negatively. Thus, assigning a general category to these less frequent values helps to keep the robustness of the model.

it is a good option to unite the labels with a count less than 100 to a new category like “Other”.

In [10]:
binning_summary = get_data_frame_summmary(df, 
                                    sort_by='unique_counts', 
                                    ascending=True)   

### Columns with less than 100 unique categories

In [11]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(binning_summary[binning_summary['unique_counts'] < 100])
    

,index,unique_values,unique_counts,data_type,percent_missing


In [12]:
columns_to_bin = binning_summary[binning_summary['unique_counts'] < 100]['index'].to_list()

In [13]:
# select columns with less than 100 labels
df = df.apply(lambda x: x.mask(x.map(x.value_counts())<10, 'Other') if x.name in columns_to_bin else x)

### After Binning - Columns with less than 100 unique categories

In [14]:
binning_summary = get_data_frame_summmary(df, 
                                    sort_by='unique_counts', 
                                    ascending=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(binning_summary[binning_summary['unique_counts'] < 100])

,index,unique_values,unique_counts,data_type,percent_missing


## Log Transform

Logarithm transformation (or log transform) is one of the most commonly used mathematical transformations

It helps to handle skewed data and after transformation, the distribution becomes more approximate to normal.
In most of the cases the magnitude order of the data changes within the range of the data. log transform normalizes the magnitude differences.

It also decreases the effect of the outliers, due to the normalization of magnitude differences and the model become more robust.

Important note: The data you apply log transform must have only positive values, otherwise you receive an error. Also, you can add 1 to your data before transform it. Thus, you ensure the output of the transformation to be positive.


In [15]:
# No Log transform applied 

## Scaling

In most cases, the numerical features of the dataset do not have a certain range and they differ from each other. Scaling solves this problem. The continuous features become identical in terms of the range, after a scaling process. This process is not mandatory for many algorithms, but it might be still nice to apply. 

### Normalization

Normalization (or min-max normalization) scale all values in a fixed range between 0 and 1. This transformation does not change the distribution of the feature and due to the decreased standard deviations, the effects of the outliers increases. Therefore, before normalization, it is recommended to handle the outliers

### Standardization
Standardization (or z-score normalization) scales the values while taking into account standard deviation. If the standard deviation of features is different, their range also would differ from each other. This reduces the effect of the outliers in the features.
In the following formula of standardization, the mean is shown as μ and the standard deviation is shown as σ.

In [16]:
df = scale_numeric_features(df, inplace=True)

********************* - Scaling following 3 features - **********************
 Bookigns - Committed Print(Rep) standardized to Bookigns - Committed Print(Rep)
 Cancellation Month standardized to Cancellation Month
 Reporting Year (2015) standardized to Reporting Year (2015)


## One-hot encoding

This method spreads the values in a column to multiple flag columns and assigns 0 or 1 to them. These binary values express the relationship between grouped and encoded column.

This method changes your categorical data, which is challenging to understand for algorithms, to a numerical format

If you have N distinct values in the column, it is enough to map them to N-1, as the missing value can be deduced from the other columns



In [17]:
print(columns_to_bin)

[]


In [18]:
# Specify Columns to encode
columns_to_exclude = ['Product Line Level 3','Payment Term Description','Payment Term', 
                      'Business Division (Agreement SIS)', 'Product Line Level 4']
columns_to_include = columns_to_bin
columns_to_encode = remove_items(columns_to_include, columns_to_exclude)
columns_to_encode

[]

In [19]:
for column in columns_to_encode:
    encoded_columns = pd.get_dummies(df[column])
    print(f'Encoding collumns : {column} to {len(encoded_columns.columns)} new encoded columns')
    df = df.join(encoded_columns, rsuffix='_'+column).drop(column, axis=1)

In [20]:
df.shape

(0, 37)

In [21]:
df.head()

,Source System,SIS Id (Agreement SIS),Source System.1,HQ SIS Id (Agreement SIS),Name (Agreement SIS),Business Division (Agreement SIS),Sales Division (Agreement SIS),Division,RSO,Subregion Grouping,...,Agreement Number,Agreement Start Date,Agreement End Date,Subscription Start Date,Subscription End Date,Parent Agreement Number,Currency(Entered),Bookigns - Committed Print(Rep),Cancellation Month,Cancellation Reason


## Transform Date columns

Recency and Frequency Variables and length / duration.


In [22]:
# convert date columns to date data type

date_columns = ['Agreement Start Date', 'Agreement End Date','Subscription Start Date', 'Subscription End Date']

In [23]:
for column in date_columns:
    df[column] = pd.to_datetime(df[column])

In [24]:
# calcualte new variable subscription_length and agreement_length
df['subscription_length'] = months_between_columns(df['Subscription End Date'],df['Subscription Start Date'])

In [25]:
df['agreement_length'] = months_between_columns(df['Agreement End Date'],df['Agreement Start Date'])

In [26]:
 df.head()

,Source System,SIS Id (Agreement SIS),Source System.1,HQ SIS Id (Agreement SIS),Name (Agreement SIS),Business Division (Agreement SIS),Sales Division (Agreement SIS),Division,RSO,Subregion Grouping,...,Agreement End Date,Subscription Start Date,Subscription End Date,Parent Agreement Number,Currency(Entered),Bookigns - Committed Print(Rep),Cancellation Month,Cancellation Reason,subscription_length,agreement_length


## Remove unused and/or redundant Features

Unused features are those that don’t make sense to pass into our machine learning algorithms. such as ID columns
Features that wouldn't be available at the time of prediction, Other text descriptions

Redundant features would typically be those that have been replaced by other features that you’ve added during feature engineering


In [27]:
# @TODO
# check the columns use of 'Status Change Date', 'Renewal Exp Complete Date'
# check 'Bookigns - Committed Print(Rep)'

columns_to_remove = ['HQ SIS Id (Agreement SIS)', 'Name  (Agreement SIS)', 
                     'Status Change Date', 'Product Line Level 3', 'Product Line Level 4']

In [28]:
df = df.drop(columns_to_remove, axis=1)

In [29]:
df.head()

,Source System,SIS Id (Agreement SIS),Source System.1,Business Division (Agreement SIS),Sales Division (Agreement SIS),Division,RSO,Subregion Grouping,Country Name (Agreement SIS),WIP Flag,...,Agreement End Date,Subscription Start Date,Subscription End Date,Parent Agreement Number,Currency(Entered),Bookigns - Committed Print(Rep),Cancellation Month,Cancellation Reason,subscription_length,agreement_length


Save clean data to hdf store

In [30]:
store = HDFStore(path_to_clean_hdf_datastore)
store.put(key= cfg['cancellations_file']+"_clean", value=df)
store.close()
